In [ ]:
#procedure:
# 1: find all changesets in a given time 
# 2: filter the changesets for a given hashtag. ex. the hastag of the mapathon
# 3: get all users creating at least one changeset with the mapathon-hashtag within the changesets
# 4: find all changesets of the users within the given time
# 5: OPTIONAL?? filter the changesets of the user for ex. the hotosm-hashtag
# 6: extract the number of buildings created and changed within the filtered changesets of the user

#not done yet
# 7: extract all roads within the filtered changesets of the user
# 8: calculate the distance of all roads


import requests
import xml.etree.ElementTree as ET
import sys
import time
import datetime

# returns a datetimeobject representing the date/time of the oldest changeset in the ElementTree
def getdatetimeoldestchangeset(root):
    timestring = ""
    for changeset in root:
        timestring = changeset.get("created_at")

    return datetime.datetime.strptime(timestring, "%Y-%m-%dT%H:%M:%SZ") #string nach datetime 


# returns a set with all users used a given hashtag or comment in a changeset in the ElementTree
def getparticipants(root, searchhashtag):
    participantsset = set()
    for changeset in root:
        if checkForHashtag(changeset, searchhashtag):
            
            timestring = changeset.get("created_at")
            timeofChangeset = datetime.datetime.strptime(timestring, "%Y-%m-%dT%H:%M:%SZ") #string nach datetime
            
            if timeofChangeset > mapathonStart:
                participantsset.add(changeset.get('user'))
      
    return participantsset

# Checks if a certain hashtag is either in comment-tag or hashtag-tag
def checkForHashtag(changeset, searchhashtag):
    changesetHashtags = changeset.findall(".//*[@k='hashtags']")
    changesetComment = changeset.findall(".//*[@k='comment']")
        
    if not changesetHashtags:
        if not changesetComment:
            return False
        if searchhashtag in changesetComment[0].get('v').lower(): #ex JOSM writes hashtags in comments-tag instead of the hashtag-tag
            return True
        return False

    if searchhashtag in changesetHashtags[0].get('v').lower():
        return True
    
    return False


inputStart = input("Enter UTC-Starttime of Mapathon. Format: 'YYYY,mm,DD,HH' (optional)")
timespan = int(input("Duration of Mapathon in Minutes: "))
hashtag = input("Hashtag of Mapathon (optional): ").lower()

if not hashtag:
    hashtag = '#hotosm-project' # if no hastag provided show all hotosm changeset

if inputStart:
    mapathonStart = datetime.datetime.strptime(inputStart, "%Y,%m,%d,%H")
    mapathonEnd = mapathonStart + datetime.timedelta(minutes=timespan)
else:
    mapathonEnd = datetime.datetime.now() # Current time
    mapathonStart = mapathonEnd - datetime.timedelta(minutes=timespan)


d0 = mapathonEnd
participants = set()

while d0 > mapathonStart:
    
    timeparameter = f"&time=2001-01-01,{d0.strftime('%Y-%m-%dT%H:%M:%SZ')}"
    
    changesets_url = str.format("https://api.openstreetmap.org/api/0.6/changesets?closed=true{}", timeparameter) #https://help.openstreetmap.org/questions/23089/how-to-query-all-changesets-by-a-user-since-eternity
    print(changesets_url)
    
    root = ET.XML(requests.get(changesets_url).content)
    
    d0 = getdatetimeoldestchangeset(root)
    
    participants = participants.union(getparticipants(root, hashtag))

        
print (participants)



In [ ]:
# returns a list with all ChangsetID of changesets used a hashtag containing '#hotosm-project'
def getchangesets(root):
    listchangeset = []
    for changeset in root:
        if checkForHashtag(changeset, '#tm-project'):
            listchangeset.append(changeset.get("id"))
            
    return listchangeset


def getBuildingsOfChangeset(changesetId):
    changeset_url = f"https://api.openstreetmap.org/api/0.6/changeset/{changesetId}/download"
    print (changeset_url)
    rootchangeset = ET.XML(requests.get(changeset_url).content)
    countBuilding=0

    for elements in rootchangeset:
        if elements.findall(".//*[@k='building']"):
            #building = elements.findall(".//*[@k='building']")
            #print (building)
            #print (building[0].get('v'))
            countBuilding+=1
    
    return countBuilding
    
print (f"#Buildings maped between {mapathonStart} and {mapathonEnd}")
    
for particpant in participants:
    
    print("----------------------")
    
    changesetlist = []
    
    timeparameter = f"&time={mapathonStart.strftime('%Y-%m-%dT%H:%M:%SZ')},{mapathonEnd.strftime('%Y-%m-%dT%H:%M:%SZ')}"
    usernameparameter = particpant.replace(' ', '%20')
    particpant_url = f"https://api.openstreetmap.org/api/0.6/changesets?display_name={usernameparameter}{timeparameter}"
    print(particpant_url)
    
    root = ET.XML(requests.get(particpant_url).content)
    
    listchangeset= getchangesets(root)
    print (particpant)
    
    buildings=0
    for changesetId in listchangeset:
        buildings += getBuildingsOfChangeset(changesetId)
    print (buildings)


# changesetlist += getchangesets(root)